## Especialização em Inteligência Artificial – IFMG
### Recuperação da Informação: Atividade 3: Coletor + Indexador + Buscador

**Aluno**: Alexandre Fortes Santana  

**Professor**: Moisés Ramos

[Notebook: https://github.com/alexandrefortes/recuperacao-de-dados/blob/main/coletor-indexador-ri-v2/coletor-indexador-ri.ipynb](https://github.com/alexandrefortes/recuperacao-de-dados/blob/main/coletor-indexador-ri/coletor-indexador-ri.ipynb)

#### Escopo
- Coletar o conteúdo do portal da Efí Bank (https://sejaefi.com.br);
- Utilizar Scrapy;
- Para fins didáticos, ignorar contepudos da '/central-de-ajuda' e do /blog;
- Armazenar os dados em .json;
- Mecanismo para minimizar a sobrecarga no portal;
- A partir de todo o conteúdo coletado, implementar o indexador sob a abordagem de uma lista invertida (também chamada de índice invertido).
- Implementar uma forma de avaliar a força da palavra ao indexá-la na base de dados, ou seja, o quanto ela é rara na base, ou frequente em cada página, ou a relevância dela no conjunto de palavras de cada página, etc. 
- Implementar alguma forma para detectar a codificação das palavras e tratar isso durante a indexação (lembrando que essa forma de tratamento deve aparecer depois ao tratar as palavras de pesquisa).
- Modelagem e Processamento de Consultas
    - Modelos de Correspondência de Consultas: Implemente modelos como Booleano, Vetorial, BM25 ou probabilístico clássico (BM1) para avaliar a similaridade entre uma consulta e os documentos no índice. Esses modelos ajudam a determinar quais documentos são mais relevantes para uma consulta fornecida.
    - PageRank: Utilize o algoritmo de PageRank para avaliar a importância de uma página baseada na estrutura de links da Web. O PageRank considera uma página mais importante se ela for referenciada por muitas outras páginas, especialmente aquelas que são, por si só, consideradas importantes.
    - HITS (Hubs and Authorities): Implemente o algoritmo HITS para identificar hubs e autoridades dentro do conjunto de documentos. Hubs são páginas que referenciam muitas autoridades (páginas consideradas importantes sobre um assunto específico), e autoridades são as páginas referenciadas pelos hubs.

# Coletor e Indexador

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
import subprocess
import sys

def run_spider(spider_file):
    # Executa em um processo separado.
    
    cmd = [sys.executable, spider_file]
    
    # Executa o comando e espera até que o processo seja concluído
    process = subprocess.run(cmd, capture_output=True, text=True)
    
    # Imprime o output do processo
    print(process.stdout)
    
    # Verifica se ocorreu algum erro
    if process.stderr:
        print("Erros encontrados:")
        print(process.stderr)

run_spider('efi_crawl_spider.py')


Errores encontrados:
2024-04-07 19:13:37 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
2024-04-07 19:13:37 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.9.0, w3lib 2.1.2, Twisted 24.3.0, Python 3.8.19 (default, Mar 20 2024, 19:55:45) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Windows-10-10.0.22621-SP0
2024-04-07 19:13:37 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-07 19:13:37 [py.warnings] WARNING: c:\Users\alexa\anaconda3\envs\efi_spider\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See

# Buscador (processamento de consultas do usuário) 

- Carregamento do Índice Invertido e TF-IDFa partir do arquivo JSON gerado pela pipeline TfidfCalculationPipeline.
- Função para processar a consulta do usuário, seguindo os mesmos passos de pré-processamento realizados pela pipeline TfidfCalculationPipeline.
    - Tokenizar a consulta, remover as stopwords, lematizar as palavras restantes usando o spaCy e obter uma lista de termos da consulta.
- Função para calcular a pontuação de relevância dos documentos em relação à consulta. Calcular a soma das pontuações TF-IDF dos termos da consulta presentes em cada documento.  

Se der tempo: implementar algoritmos de ranking, como o BM25 ou o algoritmo de pesquisa vetorial.
- Ordenar os documentos por ordem decrescente de pontuação de relevância.
- Apresentação dos Resultados: Exibir os títulos e URLs.

In [14]:
import json
import spacy

# Carregar o modelo do spaCy
nlp = spacy.load('pt_core_news_sm')

def load_tfidf_index(file_path='tfidf_index.json'):
    """Carrega o índice TF-IDF do arquivo JSON especificado."""
    with open(file_path, 'r', encoding='utf-8') as file:
        tfidf_index = json.load(file)
    return tfidf_index

def process_query(query):
    """Processa a consulta do usuário usando o spaCy."""
    doc = nlp(query.lower())
    filtered_tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return filtered_tokens

def calculate_relevance_scores(processed_query, tfidf_index):
    """Calcula as pontuações de relevância para cada documento."""
    scores = {}
    for term in processed_query:
        for doc_url, terms_scores in tfidf_index.items():
            if term in terms_scores:
                if doc_url not in scores:
                    scores[doc_url] = 0
                scores[doc_url] += terms_scores[term]
    
    sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)
    return sorted_scores

def load_collected_data(file_path='output-pipeline-completo.json'):
    """Carrega os dados coletados do arquivo JSON especificado."""
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    return data

def display_results(sorted_scores, collected_data):
    """Exibe os resultados, mostrando os títulos e URLs dos documentos com score maior que zero."""
    url_to_data = {item['url']: item for item in collected_data}
    filtered_scores = [score for score in sorted_scores if score[1] > 0]  # Filtra scores maiores que zero

    if not filtered_scores:  # Verifica se há resultados para mostrar
        print("Nenhum resultado encontrado.")
        return

    for url, score in filtered_scores: 
        data = url_to_data.get(url, {})
        title = data.get('main_title', 'No title available')
        print(f"Title: {title}\nURL: {url}\nScore: {score:.2f}\n")


# Exemplo de uso completo
# Carregar o índice TF-IDF e os dados coletados
tfidf_index = load_tfidf_index()
collected_data = load_collected_data()

# Processar uma consulta do usuário
query = "birôs de crédito"
processed_query = process_query(query)

# Calcular e exibir as pontuações de relevância
sorted_scores = calculate_relevance_scores(processed_query, tfidf_index)
display_results(sorted_scores, collected_data)


Title: Desenrola com a  Efí Bank:
URL: https://sejaefi.com.br/desenrola-brasil
Score: 0.03

Title: Entre na lista de espera do  Cartão de Crédito Efí
URL: https://sejaefi.com.br/solicitar-cartao-de-credito
Score: 0.02

Title: Cotação  do dólar
URL: https://sejaefi.com.br/cotacao-do-dolar
Score: 0.02

Title: Parceria: SophiA & Efí (antiga Gerencianet)
URL: https://sejaefi.com.br/formularios/sophia
Score: 0.02

Title: Termos e  contratos
URL: https://sejaefi.com.br/termos-e-contratos
Score: 0.02

Title: Efí Bank: uma Conta Digital completa para o seu negócio
URL: https://sejaefi.com.br/formularios/contate-nosso-especialista
Score: 0.01

Title: Efí Bank + IXC Soft = desconto especial para você receber por Pix, Bolix (boletos + Pix) e cartão de crédito
URL: https://sejaefi.com.br/formularios/parceria-ixc-soft
Score: 0.01

Title: Peça já sua Maquininha de Cartão da Efí
URL: https://sejaefi.com.br/formularios/solicitar-maquininha
Score: 0.01

Title: A Conta PJ com soluções de recebimento que